In [ ]:
# 0518 v1
# 새로운 방식 시도 - 좌표를 따라서 움직이는 게 아니라 마커 위치를 따라 이동
# a번 마커 위치에 도달 판단? 카메라 시야에 a번 마커가 중심에 오는지

#1. 아루코 설정 및 카메라 열기
#2. 순서대로 마커 추적
#3. 목표 마커가 카메라 중심 근처에 오면 다음 마커로 진행


In [ ]:
#!/usr/bin/env python3
import rospy
import cv2
import cv2.aruco as aruco
import numpy as np
from std_msgs.msg import Int32MultiArray

class MarkerDetectorCV:
    def __init__(self):
        rospy.init_node("marker_detector_cv_node")

        # 카메라 열기
        self.cap = cv2.VideoCapture(0)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

        if not self.cap.isOpened():
            rospy.logerr("❌ 카메라 열기 실패!")
            exit()

        # 마커 추적 순서
        self.target_ids = [12, 13, 14, 17, 16, 15]
        self.current_index = 0

        # ROS 퍼블리셔
        self.center_pub = rospy.Publisher("/marker_center", Int32MultiArray, queue_size=10)

        # 아루코 설정
        self.aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
        self.detector_params = aruco.DetectorParameters()

        rospy.loginfo("📷 OpenCV 카메라 기반 마커 인식 노드 시작")

        self.run()

    def run(self):
        rate = rospy.Rate(10)

        while not rospy.is_shutdown():
            ret, frame = self.cap.read()
            if not ret:
                rospy.logwarn("⚠️ 프레임 읽기 실패")
                continue

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            h, w = gray.shape
            center_x, center_y = w // 2, h // 2

            corners, ids, _ = aruco.detectMarkers(gray, self.aruco_dict, parameters=self.detector_params)

            if ids is not None:
                ids = ids.flatten()
                if self.current_index >= len(self.target_ids):
                    rospy.loginfo("✅ 모든 마커 인식 완료")
                    continue

                target_id = self.target_ids[self.current_index]

                if target_id in ids:
                    idx = list(ids).index(target_id)
                    corner = corners[idx][0]
                    cx = int(corner[:, 0].mean())
                    cy = int(corner[:, 1].mean())

                    # 퍼블리시: [cx, cy, target_id]
                    msg = Int32MultiArray(data=[cx, cy, target_id])
                    self.center_pub.publish(msg)

                    rospy.loginfo(f"🎯 마커 {target_id} 감지: 중심 ({cx}, {cy})")

                    # 마커 시각화
                    aruco.drawDetectedMarkers(frame, corners, ids)
                    cv2.circle(frame, (cx, cy), 6, (0, 0, 255), -1)
                    cv2.line(frame, (cx, cy), (center_x, center_y), (255, 0, 0), 2)

            cv2.imshow("Marker Detector", frame)
            if cv2.waitKey(1) & 0xFF == 27:
                break

            rate.sleep()

        self.cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    try:
        MarkerDetectorCV()
    except rospy.ROSInterruptException:
        pass
